In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

from tensorflow import keras

print(tf.__version__) # tutorial uses tensorflow version 2.0.0

2.0.0


In [10]:
# reads entire csv file and convert to np array
data = pd.read_excel(io="./data/formatted/processedData201819.xlsx")
data = np.array(data)
databody = data[:, 2:].copy()
# data - swap team 1 and 2, so it doesn't learn pattern that team 1 always win
# one row -> two teams data
r = databody.shape[0]
c= databody.shape[1]
ind = np.random.choice(r, int(r/2), replace=False)
temp = databody[ind, 0:int(c/2)]
databody[ind, 0:int(c/2)] = databody[ind, int(c/2):]
databody[ind, int(c/2):] = temp
# output -> 1 = win, 0 = lose
oury = np.ones((r, 1))
oury[ind] = 0

In [38]:
# Following https://www.tensorflow.org/tutorials/keras/classification

# Model Setup = two layers
# layer 1 - 100 neurons, sigmoid activation func
# layer 2 - 10 neurons, softmax activation func
model = keras.Sequential([
    keras.layers.Dense(500, activation='sigmoid'),
    keras.layers.Dense(10, activation='softmax')
])

# More model setup - Set optimizer, loss function, and metrics for model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Training model
# Convert np array to tensor array
tfdata = tf.convert_to_tensor(databody, np.float32)
tfout =  tf.convert_to_tensor(oury, np.float32)

model.fit(tfdata, tfout, epochs=100)

# Evaluating Accuracy
test_loss, test_acc = model.evaluate(tfdata,  tfout, verbose=2)

print('\nTest accuracy:', test_acc)

# # Make predictions
# # Need test data to make this work
# predictions = model.predict(testdata)
# print(predictions[0])

Train on 60 samples
Epoch 1/100
60/60 [==============================] - 0s 4ms/sample - loss: 1.7531 - accuracy: 0.4000
Epoch 2/100
60/60 [==============================] - 0s 116us/sample - loss: 1.0928 - accuracy: 0.5000
Epoch 3/100
60/60 [==============================] - 0s 116us/sample - loss: 0.8285 - accuracy: 0.5833
Epoch 4/100
60/60 [==============================] - 0s 100us/sample - loss: 0.7242 - accuracy: 0.5833
Epoch 5/100
60/60 [==============================] - 0s 150us/sample - loss: 0.6881 - accuracy: 0.5667
Epoch 6/100
60/60 [==============================] - 0s 116us/sample - loss: 0.6643 - accuracy: 0.6167
Epoch 7/100
60/60 [==============================] - 0s 134us/sample - loss: 0.6447 - accuracy: 0.7667
Epoch 8/100
60/60 [==============================] - 0s 166us/sample - loss: 0.6450 - accuracy: 0.6500
Epoch 9/100
60/60 [==============================] - 0s 83us/sample - loss: 0.6267 - accuracy: 0.6667
Epoch 10/100
60/60 [==============================] - 0s

60/60 [==============================] - 0s 116us/sample - loss: 0.5526 - accuracy: 0.8167
Epoch 80/100
60/60 [==============================] - 0s 183us/sample - loss: 0.5451 - accuracy: 0.8667
Epoch 81/100
60/60 [==============================] - 0s 100us/sample - loss: 0.5392 - accuracy: 0.8500
Epoch 82/100
60/60 [==============================] - 0s 150us/sample - loss: 0.5322 - accuracy: 0.8500
Epoch 83/100
60/60 [==============================] - 0s 199us/sample - loss: 0.5270 - accuracy: 0.8500
Epoch 84/100
60/60 [==============================] - 0s 133us/sample - loss: 0.5327 - accuracy: 0.7667
Epoch 85/100
60/60 [==============================] - 0s 166us/sample - loss: 0.5282 - accuracy: 0.8333
Epoch 86/100
60/60 [==============================] - 0s 166us/sample - loss: 0.5378 - accuracy: 0.8000
Epoch 87/100
60/60 [==============================] - ETA: 0s - loss: 0.5694 - accuracy: 0.71 - 0s 116us/sample - loss: 0.5471 - accuracy: 0.7667
Epoch 88/100
60/60 [===============

In [30]:

# # processing input training data
# input_train = tf.estimator.inputs.numpy_input_fn(
#                   x = {'x': databody},
#                   y = oury,
#                   num_epochs = 100,
#                   shuffle = False)

# feature_columns = [tf.feature_column.numeric_column(key='x', shape=(databody.shape[1],))]

# # makes a deep neural network with 2 hidden layers with 512 neurons each, and 10 output neurons
# dnn_clf=tf.estimator.DNNClassifier(hidden_units=[512,512], n_classes=10, feature_columns=feature_columns)

# # fits the DNN to the training data using the training labels
# dnn_clf.fit(x=databody, y=oury, batch_size=50, steps=40000)

# # evaluates the performance of the DNN on the training data
# dnn_clf.evaluate(databody, oury)